In [2]:
import sys, pdb
import logging
import argparse
import numpy as np

from scipy.stats import norm

sys.path.insert(0, '../../../network')
from network import Population, RateNetwork
from transfer_functions import ErrorFunction
from connectivity import SparseConnectivity, LinearSynapse, ThresholdPlasticityRule
from sequences import GaussianSequence

logging.basicConfig(level=logging.INFO)

In [3]:
T = 0.4

phi = ErrorFunction(mu=0.07/14., sigma=0.05/14.).phi
exc = Population(N=40000, tau=1e-2, phi=phi)

P, S = 1, 30
sequences = [GaussianSequence(S,exc.size,seed=i) for i in range(P)]
patterns = np.stack([s.inputs for s in sequences])

conn_EE = SparseConnectivity(source=exc, target=exc, p=0.005)
plasticity = ThresholdPlasticityRule(x_f=1.645, q_f=0.8)
synapse = LinearSynapse(conn_EE.K, A=1)
conn_EE.store_sequences(patterns, synapse.h_EE, plasticity.f, plasticity.g)

net = RateNetwork(exc, c_EE=conn_EE, formulation=1)

r0 = exc.phi(plasticity.f(patterns[0,0,:]))
net.simulate(T, r0=r0)

M = np.mean(net.exc.state**2, axis=0)

correlations = sequences[0].overlaps(net, exc, plasticity=plasticity, correlation=True)
overlaps = sequences[0].overlaps(net, exc, plasticity=plasticity, correlation=False)
state = np.copy(net.exc.state).astype(np.float16)

logging.info("Saving data")
np.save("data/data_bcd.npy", [state, correlations, overlaps])

INFO:connectivity:Building connections from exc to exc
INFO:connectivity:Storing sequences
100%|██████████| 1/1 [00:02<00:00,  2.93s/it]675.20it/s]
INFO:connectivity:Applying synaptic transfer function
INFO:connectivity:Building sparse matrix
INFO:network:Integrating network dynamics
[========================================================================] 100%


--Return--
> <ipython-input-3-dbba97b7df22>(21)<module>()->None
-> pdb.set_trace()


(Pdb)  c


INFO:sequences:Computing correlations
100%|██████████| 30/30 [00:15<00:00,  1.97it/s]
INFO:sequences:Computing overlaps
100%|██████████| 30/30 [00:00<00:00, 151.65it/s]
INFO:root:Saving data
